In [4]:
# counting total words and unique words
import re
from collections import Counter

def count_words(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
    except FileNotFoundError:
        print(f"Error: '{file_path}' not found.")
        return

    # 🔹 Clean the text: remove punctuation & normalize spaces
    text = re.sub(r'[“”"\'’‘.,!?;:()\[\]{}]', ' ', text)
    text = re.sub(r'\s+', ' ', text.strip())

    # 🔹 Split into tokens (words)
    words = text.split(' ')   # Faster than regex split for simple whitespace

    # 🔹 Count all words using Counter (highly efficient)
    word_counts = Counter(words)

    # 🔹 Compute metrics
    total_words = sum(word_counts.values())      # includes duplicates
    unique_words = len(word_counts)              # only unique ones

    print(f"Total words (including duplicates): {total_words}")
    print(f"Total unique words: {unique_words}")

    # 🔹 Optional: print 10 most common words
    print("\nTop 10 most frequent words:")
    for word, freq in word_counts.most_common(10):
        print(f"{word}: {freq}")

# ------------------- Run -------------------
count_words("marathi_clean.txt")


Total words (including duplicates): 385259
Total unique words: 46067

Top 10 most frequent words:
आहे: 11564
यांनी: 5680
आणि: 3415
पवार: 2596
या: 2541
आहेत: 2517
केली: 2455
केला: 1925
नाही: 1873
ठाकरे: 1596


In [ ]:
# Install required packages (if running in Colab/Jupyter)
!pip install pandas regex scikit-learn tensorflow -q

In [1]:
import pandas as pd
import regex as re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

2025-10-04 09:12:49.529714: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-04 09:12:49.593812: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-04 09:12:51.438124: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


In [2]:
try:
    with open("temp.txt", "r", encoding="utf-8") as file:
        text = file.read()
except FileNotFoundError:
    print("Error: 'temp.txt' not found.")
    text = ""

In [8]:
if text:
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts([text])
    total_words = len(tokenizer.word_index) + 1
    print("total unique words ",total_words)
    # Convert entire corpus into one sequence of tokens
    token_list = tokenizer.texts_to_sequences([text])[0]
    print("total words in corpus ",len(token_list))

total unique words  3269
total words in corpus  6758


In [ ]:
input_sequences = []
window_size = 10   # you can tune this (context length)
for i in range(len(token_list) - window_size):
    seq = token_list[i : i + window_size]   # context
    label = token_list[i + window_size]     # next word
    input_sequences.append((seq, label))
    
# input_sequences = [
#   ([12, 45, 78], 102),
#   ([45, 78, 102], 56)
# ]


In [15]:
# Separate predictors (x) and labels (y)
x = np.array([seq for seq, label in input_sequences])
y = np.array([label for seq, label in input_sequences])

x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=42
    )
max_sequence_len = x.shape[1]  # window_size
print(max_sequence_len)


10


In [ ]:
# -------------------- 3. Model --------------------
model = Sequential()
model.add(
    Embedding(
        total_words, #3269 in temp
        128, # dimension of each word vector
        input_length=max_sequence_len, # window size 10
        embeddings_regularizer=regularizers.l2(1e-6)  #Embedding regularization
    )
)
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(LSTM(64))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(total_words, activation="softmax"))
# Manually build
model.build(input_shape=(None, max_sequence_len))

# ✅ Sparse loss = no one-hot
model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        optimizer="adam",
        metrics=["accuracy"]
    )

model.summary()




Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 10, 128)        │       418,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 10, 256)        │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 10, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        82,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3269)           │       421,701 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,193,797 (4.55 MB)

 Trainable params: 1,193,797 (4.55 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# -------------------- 4. Training --------------------
early_stopping = EarlyStopping(
        monitor='val_loss', patience=5, restore_best_weights=True
    )

print("\n--- Starting Model Training ---")
history = model.fit(
        x_train,
        y_train,
        epochs=50,
        validation_data=(x_test, y_test),
        verbose=1,
        callbacks=[early_stopping]
    )
print("\n--- Model Training Complete ---")


--- Starting Model Training ---
Epoch 1/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - accuracy: 0.0274 - loss: 7.8708 - val_accuracy: 0.0370 - val_loss: 7.7419
Epoch 2/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.0298 - loss: 7.3762 - val_accuracy: 0.0370 - val_loss: 7.9151
Epoch 3/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.0298 - loss: 7.1595 - val_accuracy: 0.0348 - val_loss: 8.0086
Epoch 4/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.0295 - loss: 6.9812 - val_accuracy: 0.0370 - val_loss: 8.3103
Epoch 5/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - accuracy: 0.0306 - loss: 6.8043 - val_accuracy: 0.0341 - val_loss: 8.4946
Epoch 6/50
169/169 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.0343 - loss: 6.5974 - val_accuracy: 0.0356 - val_loss: 8.7964

--- Model Training Complete ---
